In [4]:
pip install opencv-python

In [1]:
import face_recognition
#from sklearn import svm
import os
import cv2
import numpy as np

In [2]:
import argparse
import pickle
import csv
from datetime import datetime
    

In [18]:
encodings = []
names = []

# Training directory
train_dir = os.listdir('C:/Users/Gourab/Downloads/wahida rehman/training')

# Loop through each person in the training directory
for person in train_dir:
    pix = os.listdir("C:/Users/Gourab/Downloads/wahida rehman/training/" + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("C:/Users/Gourab/Downloads/wahida rehman/training/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

Gourab/gourab24.jpg was skipped and can't be used for training


In [19]:
print(names)

['chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'chad smith', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'Gourab', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel', 'will ferrel']


In [4]:
# serialising the encodings
data ={"encodings":encodings,"names":names}
file_to_write = open("face_pkl2.pkl", "wb")
pickle.dump(data, file_to_write)

In [9]:
file_to_read = open("face_pkl2.pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
print(loaded_dictionary)

{'encodings': [array([-0.07208099,  0.14363752,  0.0411058 , -0.11355335, -0.09391195,
       -0.08291854, -0.04923932, -0.05358117,  0.13825043, -0.03793276,
        0.24703257, -0.04216716, -0.19422935, -0.03559211, -0.08763224,
        0.09544582, -0.09033161,  0.03354545, -0.13590233, -0.17275865,
       -0.03260054,  0.0379913 ,  0.04358697,  0.02793899, -0.09196095,
       -0.26615149, -0.04429611, -0.07285438,  0.03669097, -0.13010941,
        0.04493492,  0.02969311, -0.17653278, -0.08266571,  0.01335683,
        0.06001599, -0.16913749, -0.09662644,  0.2263962 , -0.01951276,
       -0.0337683 ,  0.03664764,  0.07026815,  0.27323744,  0.18679649,
       -0.04635335,  0.03146806, -0.11195733,  0.21201776, -0.19537243,
        0.01243172,  0.15813121,  0.11373901,  0.09571339,  0.06717232,
       -0.09480172,  0.08972242,  0.14226909, -0.25085855, -0.00628129,
        0.03907829, -0.01116178, -0.14653048, -0.02441563,  0.19802281,
        0.16917472, -0.13049954, -0.1176163 ,  0.

In [31]:
names=list(loaded_dictionary.get('names'))

In [20]:
encodings=list(loaded_dictionary.get('encodings'))

In [13]:
print(getList(loaded_dictionary))

['encodings', 'names']


In [26]:
with open('ATTLOG.csv','w') as f:
    pass

In [27]:
def MarkAttendance(name):
    already_in_file = set()
    with open('ATTLOG.csv', "r+") as g: 
        # just read
        for line in g:
            already_in_file.add(line.split(",")[0])

# process the current entry:
    if name not in already_in_file:
        with open('ATTLOG.csv', "a") as g:
            # append
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            g.writelines(f'\n{name},{dtString}')

In [30]:
# Initialize some variables
video_capture = cv2.VideoCapture(0)
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        MarkAttendance(name)
        
  

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()